In [27]:
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mandelbrot import mandelbrot, mandelbrot_matrix, complex_matrix
from monte_carlo_integration import mc_int_estimate

C:\Users\arend\anaconda3\envs\zeearend\envs\zeearend\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [ ]:
def complex_matrix(xmin, xmax, ymin, ymax, pixel_density):
    re = np.linspace(xmin, xmax, int((xmax - xmin) * pixel_density))
    im = np.linspace(ymin, ymax, int((ymax - ymin) * pixel_density))
    return re[np.newaxis, :] + im[:, np.newaxis] * 1j

In [4]:
def mandelbrot_matrix(matrix, iterations):
    for i in range(np.shape(matrix)[0]):
        for j in range(np.shape(matrix)[1]):
            value = mandelbrot(matrix[i][j], iterations)
            matrix[i][j] = value
    return matrix

def mandelbrot(c, iterations):
    z = 0
    for _ in range(iterations):
        z = z**2 + c
    return z >= -2 and z <= 2

In [54]:
mat = complex_matrix(-2, 1, -1, 1, 100)

In [ ]:
for i in range(np.shape(mat)[0]):
    for j in range(np.shape(mat)[1]):
        v = mandelbrot(mat[i][j], 100)
        mat[i][j] = v

<ipython-input-45-31573ef9bf53>:8: RuntimeWarning: overflow encountered in cdouble_scalars
  z = z**2 + c
<ipython-input-45-31573ef9bf53>:8: RuntimeWarning: invalid value encountered in cdouble_scalars
  z = z**2 + c


In [ ]:
sns.heatmap(np.real(mat), xticklabels=0, yticklabels =0, cbar=False, square=True)
plt.show()

In [ ]:
def mc_int_estimate(iterations, samples):
    matrix = complex_matrix(-2, 1, -1, 1, 100)
    mandelbrot = mandelbrot_matrix(matrix, iterations)
    in_mandelbrot = 0
    for i in range(samples):
        if np.real(mandelbrot[np.random.randint(np.shape(mandelbrot)[0]), np.random.randint(np.shape(mandelbrot)[1])]):
            in_mandelbrot += 1
    return in_mandelbrot/samples * ((1- -2)* (1- - 1))

In [ ]:
l = []
for i in np.linspace(50, 500, 12):
    l.append(mc_int_estimate(int(i), 50))
plt.plot(l)
